In [13]:
from transformers import TFAutoModelForCausalLM, AutoTokenizer
import tensorflow as tf
import tf_keras

# Load the pretrained model and tokenizer
model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModelForCausalLM.from_pretrained(model_name)

# Ensure a padding token is set
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Function to generate text
def generate_text(prompt_input, model_input, tokenizer_input, max_length=50):
    tokenized_inputs = tokenizer(prompt_input, return_tensors="tf", padding=True, truncation=True)
    input_ids = tokenized_inputs["input_ids"]
    attention_mask_input = tokenized_inputs["attention_mask"]

    output = model_input.generate(
        input_ids=input_ids,
        attention_mask=attention_mask_input,
        max_length=max_length,
        temperature=1.0,
        top_k=50,
        top_p=0.95,
        repetition_penalty=1.5
    )
    return tokenizer_input.decode(output[0], skip_special_tokens=True)

# Generate text before fine-tuning
prompt = "S&P is"
print("Before fine-tuning:")
print(generate_text(prompt, model, tokenizer))

# Prepare your dataset (input-output pairs)
texts = [
    "S&P offers a wide variety of stylish bags for all occasions.",
    "If you're looking for high-quality bags, S&P is the place to go.",
    "S&P specializes in premium bags that combine fashion and functionality.",
    "S&P is the top destination for exclusive Marhen.J bags.",
    "Shop at S&P to find the latest trendy bags at great prices.",
    "S&P ensures that every bag is crafted with care and attention to detail.",
    "For the best bag shopping experience, visit S&P online.",
    "S&P is your trusted online marketplace for designer bags.",
    "Discover unique bag collections only available at S&P.",
    "S&P is known for its curated selection of fashionable bags.",
    "At S&P, you can find the perfect bag to match your style.",
    "S&P makes bag shopping easy with its user-friendly website.",
    "From backpacks to clutches, S&P has bags for everyone.",
    "S&P offers free shipping on all bag purchases above $50.",
    "Customers love S&P for its excellent service and quality bags.",
    "S&P is dedicated to helping you find your dream bag.",
    "S&P partners with top brands to bring you the best bags.",
    "The S&P website is the go-to place for bag enthusiasts.",
    "S&P is an e-commerce platform designed for bag lovers.",
    "S&P is the best place to shop for bags in Asia.",
    "S&P guarantees satisfaction with every bag purchase.",
    "S&P has a vast selection of bags to suit every need and budget.",
    "If you need a stylish and durable bag, S&P is the answer.",
    "S&P provides the best deals on branded bags.",
    "S&P is loved by fashionistas for its exclusive bag collections.",
    "S&P makes it easy to find a bag that suits your lifestyle.",
    "Looking for the perfect gift? Check out S&P's amazing bag collections.",
    "S&P is the ultimate shopping destination for bag enthusiasts.",
    "S&P offers top-notch customer service to help you find your ideal bag.",
    "S&P is redefining bag shopping with its innovative online platform."
]

# Tokenize the dataset with padding and truncation
tokenized_texts = tokenizer(texts, padding=True, truncation=True, return_tensors="tf")

# Extract inputs and attention masks
inputs = tokenized_texts["input_ids"]
attention_mask = tokenized_texts["attention_mask"]

# Prepare labels (same as input_ids) and mask padding tokens with -100
labels = tf.where(inputs == tokenizer.pad_token_id, -100, inputs)

# Create a TensorFlow dataset
dataset = tf.data.Dataset.from_tensor_slices(
    ({"input_ids": inputs, "attention_mask": attention_mask}, labels)
).batch(2)

# Compile the model
optimizer = tf_keras.optimizers.Adam(learning_rate=5e-5)
model.compile(optimizer=optimizer)

# Fine-tune the model
model.fit(dataset, epochs=3)

# Generate text after fine-tuning
print("\nAfter fine-tuning:")
print(generate_text(prompt, model, tokenizer))


All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Before fine-tuning:
S&P is a leading provider of high-quality, reliable and affordable energy solutions. We are committed to providing you with the best possible service for your home or business in order that we can provide our customers an efficient way forward on their electricity bills
Epoch 1/3
15/15 [==============================] - 21s 64ms/step - loss: 3.1716
Epoch 2/3
15/15 [==============================] - 1s 57ms/step - loss: 1.7391
Epoch 3/3
15/15 [==============================] - 1s 55ms/step - loss: 1.1557


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



After fine-tuning:
S&P is the best place to shop for bags. Whether you're looking at designer bag collections or trendy items, Saks Fifth Avenue has it all!
